## Building neural network with Keras

In [1]:
#Importing the dataset 

import csv
import pandas as pd

# add header names
headers =  ['age', 'sex','chest_pain','resting_blood_pressure',  
        'serum_cholestoral', 'fasting_blood_sugar', 'resting_ecg_results',
        'max_heart_rate_achieved', 'exercise_induced_angina', 'oldpeak',"slope of the peak",
        'num_of_major_vessels','thal', 'heart_disease']

heart_df = pd.read_csv('heart.dat', sep=' ', names=headers)
heart_df.head()

,age,sex,chest_pain,resting_blood_pressure,serum_cholestoral,fasting_blood_sugar,resting_ecg_results,max_heart_rate_achieved,exercise_induced_angina,oldpeak,slope of the peak,num_of_major_vessels,thal,heart_disease
0,70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0,3.0,2
1,67.0,0.0,3.0,115.0,564.0,0.0,2.0,160.0,0.0,1.6,2.0,0.0,7.0,1
2,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0,2
3,64.0,1.0,4.0,128.0,263.0,0.0,0.0,105.0,1.0,0.2,2.0,1.0,7.0,1
4,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,1


In [2]:
#Getting shape of the data
heart_df.shape

(270, 14)

In [3]:
#Looking for the null values through the dataset because neural net require non null variables
heart_df.isna().sum()

age                        0
sex                        0
chest_pain                 0
resting_blood_pressure     0
serum_cholestoral          0
fasting_blood_sugar        0
resting_ecg_results        0
max_heart_rate_achieved    0
exercise_induced_angina    0
oldpeak                    0
slope of the peak          0
num_of_major_vessels       0
thal                       0
heart_disease              0
dtype: int64

In [4]:
#Splitting the dataset into training and test parts
import numpy as np
import warnings
warnings.filterwarnings("ignore") #suppress warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#convert input to numpy arrays
X = heart_df.drop(columns=['heart_disease'])

#replace target class with 0 and 1 
#1 means "have heart disease" and 0 means "do not have heart disease"
heart_df['heart_disease'] = heart_df['heart_disease'].replace(1, 0)
heart_df['heart_disease'] = heart_df['heart_disease'].replace(2, 1)

y_label = heart_df['heart_disease'].values.reshape(X.shape[0], 1)

#split data into train and test set
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y_label, test_size=0.2, random_state=2)

#standardize the dataset
sc = StandardScaler()
sc.fit(Xtrain)
Xtrain = sc.transform(Xtrain)
Xtest = sc.transform(Xtest)

print(f"Shape of train set is {Xtrain.shape}")
print(f"Shape of test set is {Xtest.shape}")
print(f"Shape of train label is {ytrain.shape}")
print(f"Shape of test labels is {ytest.shape}")

Shape of train set is (216, 13)
Shape of test set is (54, 13)
Shape of train label is (216, 1)
Shape of test labels is (54, 1)


### Creating neural net with python libraries

In [5]:
#With scikit learn - with multilayer perceptron classifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

sknet = MLPClassifier(hidden_layer_sizes=(8), learning_rate_init=0.001, max_iter=100)

In [6]:
sknet.fit(Xtrain, ytrain)
preds_train = sknet.predict(Xtrain)
preds_test = sknet.predict(Xtest)

print("Train accuracy of sklearn neural network: {}".format(round(accuracy_score(preds_train, ytrain),2)*100))
print("Test accuracy of sklearn neural network: {}".format(round(accuracy_score(preds_test, ytest),2)*100))


Train accuracy of sklearn neural network: 80.0
Test accuracy of sklearn neural network: 78.0


In [7]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(ytest,preds_test)
print(cm)

[[25  7]
 [ 5 17]]


### Building NN with Keras

In [8]:
#With Keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# define the model
model = Sequential()
model.add(Dense(8, input_shape=(13,)))
model.add(Dense(1, activation='sigmoid'))

In [9]:
# compile the model
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(Xtrain, ytrain, epochs=100, verbose=1)
train_acc = model.evaluate(Xtrain, ytrain, verbose=1)[1]
# print(train_acc)
test_acc = model.evaluate(Xtest, ytest, verbose=1)[1]
# print(test_acc)
print("Train accuracy of keras neural network: {}".format(round((train_acc * 100), 2)))
print("Test accuracy of keras neural network: {}".format(round((test_acc * 100),2)))

Epoch 1/100
7/7 [==============================] - 1s 2ms/step - loss: 1.1468 - accuracy: 0.2917
Epoch 2/100
7/7 [==============================] - 0s 2ms/step - loss: 1.0854 - accuracy: 0.3056
Epoch 3/100
7/7 [==============================] - 0s 2ms/step - loss: 1.0279 - accuracy: 0.3519
Epoch 4/100
7/7 [==============================] - 0s 2ms/step - loss: 0.9752 - accuracy: 0.3750
Epoch 5/100
7/7 [==============================] - 0s 2ms/step - loss: 0.9257 - accuracy: 0.4074
Epoch 6/100
7/7 [==============================] - 0s 2ms/step - loss: 0.8781 - accuracy: 0.4213
Epoch 7/100
7/7 [==============================] - 0s 1ms/step - loss: 0.8345 - accuracy: 0.4722
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 0.7949 - accuracy: 0.5185
Epoch 9/100
7/7 [==============================] - 0s 1ms/step - loss: 0.7579 - accuracy: 0.5556
Epoch 10/100
7/7 [==============================] - 0s 1ms/step - loss: 0.7231 - accuracy: 0.5833
Epoch 11/100
7/7 [===========

7/7 [==============================] - 0s 2ms/step - loss: 0.2910 - accuracy: 0.8704
Epoch 85/100
7/7 [==============================] - 0s 1ms/step - loss: 0.2905 - accuracy: 0.8704
Epoch 86/100
7/7 [==============================] - 0s 1ms/step - loss: 0.2904 - accuracy: 0.8704
Epoch 87/100
7/7 [==============================] - 0s 1ms/step - loss: 0.2899 - accuracy: 0.8704
Epoch 88/100
7/7 [==============================] - 0s 1ms/step - loss: 0.2896 - accuracy: 0.8704
Epoch 89/100
7/7 [==============================] - 0s 1ms/step - loss: 0.2894 - accuracy: 0.8704
Epoch 90/100
7/7 [==============================] - 0s 1ms/step - loss: 0.2890 - accuracy: 0.8704
Epoch 91/100
7/7 [==============================] - 0s 1ms/step - loss: 0.2887 - accuracy: 0.8704
Epoch 92/100
7/7 [==============================] - 0s 1ms/step - loss: 0.2885 - accuracy: 0.8704
Epoch 93/100
7/7 [==============================] - 0s 2ms/step - loss: 0.2884 - accuracy: 0.8750
Epoch 94/100
7/7 [===============

In [14]:
# make probability predictions with the model
predictions = model.predict(Xtest)
print(predictions[:5])

2/2 [==============================] - 0s 2ms/step
[[0.7250462 ]
 [0.14405821]
 [0.62287956]
 [0.9901848 ]
 [0.04885133]]


In [15]:
# round predictions 
y_pred = [round(x[0]) for x in predictions]
print(y_pred)

[1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]


In [17]:
from sklearn.metrics import confusion_matrix , classification_report
print(classification_report(ytest,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.78      0.79        32
           1       0.70      0.73      0.71        22

    accuracy                           0.76        54
   macro avg       0.75      0.75      0.75        54
weighted avg       0.76      0.76      0.76        54



In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test,y_pred)

plt.figure(figsize = (10,7))
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [11]:
#How to save Keras models
#save model and architecture to single file
model.save("model_9.h5")
print("Saved model to disk")

Saved model to disk


In [12]:
#How to load saved keras model and use it again
from numpy import loadtxt
from tensorflow.keras.models import load_model
 
# load model
model = load_model("model_9.h5")
# summarize model.
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 112       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
c